In [2]:
import pandas as pd
traindf = pd.read_csv("./titanic.csv", encoding="utf-8")
testdf = pd.read_csv("./titanic.csv", encoding="utf-8")

FileNotFoundError: [Errno 2] File b'./titanic.csv' does not exist: b'./titanic.csv'

In [ ]:
traindf.isna().sum()

In [ ]:
# 1.1 填數值型的缺失
med = traindf.median()
traindf = traindf.fillna(med)
# 1.2 填類別型的缺失
most = traindf["Embarked"].value_counts().idxmax()
traindf["Embarked"] = traindf["Embarked"].fillna(most)
traindf.isna().sum()


In [ ]:
# updata test 資料 不用重算 med 
testdf = testdf.fillna(med)
testdf["Embarked"] = testdf["Embarked"].fillna(most)
testdf.isna().sum()

In [ ]:
# dummy coding Embarked sex
dummy = pd.get_dummies(traindf["Embarked"])
traindf = pd.concat([traindf, dummy], axis=1)
dummy = pd.get_dummies(traindf["Sex"])
traindf = pd.concat([traindf, dummy], axis=1)
traindf

In [ ]:
# 處理人名資訊
s = "Braund, Mr. Owen Harris"
s.split(",")[-1].split(".")[0].replace(" ", "")

In [ ]:
# 看看每一種有幾個, 再決定要留誰
def nameflow(s):
    return s.split(",")[-1].split(".")[0].replace(" ", "")
# pd.crosstab(traindf["Survived"], traindf["Name"].apply(nameflow))
# pd.crosstab(traindf["Sex"], traindf["Name"].apply(nameflow))

In [ ]:
def nameflow(s):
    s = s.split(",")[-1].split(".")[0].replace(" ", "")
    saved = ["Mr", "Mrs", "Miss"]
    if s in saved:
        return s
    else:
        return "X"
s = traindf["Name"].apply(nameflow)
dummy = pd.get_dummies(s)
traindf = pd.concat([traindf, dummy], axis=1)
traindf

In [ ]:
# 對test資料做一樣的處理
dummy = pd.get_dummies(testdf["Embarked"])
testdf = pd.concat([testdf, dummy], axis=1)
dummy = pd.get_dummies(testdf["Sex"])
testdf = pd.concat([testdf, dummy], axis=1)
s = testdf["Name"].apply(nameflow)
dummy = pd.get_dummies(s)
testdf = pd.concat([testdf, dummy], axis=1)
testdf

In [ ]:
# 3. 決定丟掉
# traindf.columns 看有哪些欄位
traindf = traindf.drop(['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'X'], axis=1)
# test
testdf = testdf.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'X'], axis=1)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# preview 個參數之間的關係
%matplotlib inline
plt.figure(figsize=(14, 14))
sns.heatmap(traindf.corr(), annot=True, cmap="RdBu")

In [ ]:
# 第一種條法: 經驗(不要使用)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x = traindf.drop(["Survived"], axis=1)
y = traindf["Survived"]
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1)
# clf = RandomForestClassifier(max_depth=7, n_estimators=27)
# clf.fit(xtrain, ytrain)
# accuracy_score(clf.predict(xtest), ytest)
# 第二種條法: 交叉驗證 做十次的平均
from sklearn.model_selection import cross_val_score
import numpy as np
clf = RandomForestClassifier(max_depth=6, n_estimators=27)
scores = cross_val_score(clf, x, y, cv=10)
print(scores)
print(np.average(scores))
print("====================================")
# 第三種條法: 讓電腦幫我對所有參數都完成一次交叉驗證, 看一下哪組參數的結果最好
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators":range(22, 26),
    "max_depth":range(4, 9)
}
clf = RandomForestClassifier()
g = GridSearchCV(clf, params, cv=10)
g.fit(x, y)
print(g.best_params_)
print(g.best_score_)

In [ ]:
clf = RandomForestClassifier(max_depth= 7, n_estimators= 23)
clf.fit(x, y)

In [ ]:
# 將完成的模型測試test
xtest = testdf.drop(["PassengerId"], axis=1)
pre = clf.predict(xtest)
resultdf = pd.DataFrame()
resultdf["PassengerId"] = testdf["PassengerId"]
resultdf["Survived"] = pre
# 存檔
resultdf.to_csv("result.csv", encoding="utf-8", index=False)
resultdf